# Extracting time series for each ROI
--------------------------------------
- This will take the already clean nifti files (created using sciprt in the /cleaning folder), and extract timecourse per ROI
- Currently using Shen parcellation, but can be changed easily


In [18]:
import pandas as pd 
from nilearn.maskers import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
from nilearn import datasets
import numpy as np
import nilearn.plotting
import os
import glob
import nilearn
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
%config Completer.use_jedi = False

In [19]:
# set output dir
output_dir = '/gpfs/gibbs/pi/levy_ifat/Or/ZivResults/timecourse/'

In [20]:
# load atlas - shen parcellation
shen = '/gpfs/gibbs/pi/levy_ifat/Or/ROI/shenPar/shen_2mm_268_parcellation.nii.gz'
#yeo = datasets.fetch_atlas_yeo_2011()

In [21]:
# create masker to extract functional data within atlas parcels
masker = NiftiLabelsMasker(labels_img=shen, standardize=True,
                           memory='nilearn_cache')

In [24]:
# grab functional files (cleaned one)
func_files = glob.glob('/gpfs/gibbs/pi/levy_ifat/Or/ZivResults/afni_cleaning_hariri/_subject_id_*/cleanAfni/sub-*_ses-1_task-hariri_space-MNI152NLin2009cAsym_desc-smoothAROMAnonaggr_bold_tproject.nii.gz')
len(func_files)

131

In [25]:
task = 'hariri' # domino or hariri
def masking(funcFile):
    sub = os.path.basename(funcFile).split('_')[0]
    # takes functional file and returns Pandas dataframe for average timeseries of each of these ROIs
    a = masker.fit_transform(funcFile)
    subDF = pd.DataFrame(a)
    # save it
    subDF.to_csv(f'{output_dir}{sub}_shen-2mm_{task}.csv', index=False)
    return subDF

In [ ]:
# run parallel
results = Parallel(n_jobs=8)(delayed(masking)(funcFile) for funcFile in func_files)